In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Cargamos nuestro fichero  de datos winequality-red.csv, el cual descargamos a traves de la plataforma Kaggle, cuyo enlace esta en en la introducción
# del ejercicio
# El delimitador de nuestro fiche es la coma, y evitamos cargar la primera line
# que corresponde al fichero

dataset= np.loadtxt('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv', delimiter=',',skiprows=1)
dataset.shape

In [ ]:
# Separamos las columnas con los datos de entrenamiento de los datos esperado
# Siendo X las primeras 11 columnas las cuales son las variables de las cuales segun la combinación de cada una de las variables da el resultadoç
# d es donde se encuentra contenida la columna del resultado  la cual es calidad que posee dicho vino
X= dataset[:, 0:11]
d = dataset[: , 11]

In [ ]:
# Verificamos que en X se encuentran las 1599 filas y las 11 columnas 
X.shape

In [ ]:
# Verificamos que en d se encuentran las 1599 filas y una unica columna
d.shape

In [ ]:
# COnvertimos una lista de valores a una matriz cuyos elementos son listas de un valor
print(d.shape)
d=np.reshape(d,(-1,1))

In [ ]:
# Vamos a aescalar nuestros datos, de cara q eu todos los datos tengan una escala similar y no tengamos datos con diferente escalado

scaler_X = MinMaxScaler()
scaler_d = MinMaxScaler()
X = scaler_X.fit_transform(X)
d = scaler_d.fit_transform(d)
d

In [ ]:
# Mostramos los datos que contiene X tras aplicarle el proceso de escalado
X

In [ ]:
# Dividimos nuestra matrix X y el vector d en datos de entrenamiento y test
from sklearn.model_selection import train_test_split
# Queremos que la muestra del test sea el 75% de nuestros datos de cara a obtener una buena cantidad,
# de datos para nuestro entrenamiento
X_train, X_test, d_train, d_test = train_test_split(X,d, test_size=0.25)

In [ ]:
# Comprobamos que efectivamente el tamaño de nuestro X_test , X_train
# las exigencias que le hemos marcado
print(X_train.shape)

In [ ]:
# Comenzamos a crear nuestro modelo 

modelo = Sequential()
# Establecemos que el input_dim=11 debido a que tenemos 11 variables a tener en cuenta para el desarrollo de nuestro modelo, luego para el activation,
# seleccione relu debido a que con la cantidad de variables que tenemos y que algunos valores son altos, en estos casos elñ más adecuado es relu, otra opción podría ser el de sigmoid
# pero con los resultados obtenidos tras las preubas, descubri que con relu se obtiene un mejor resultado a la hora de hacer la clasificación
modelo.add( Dense(12, input_dim=11, kernel_initializer='normal', activation='relu'))
# Añadimos otra oculta de con valor 5
modelo.add( Dense(5, activation='relu',kernel_initializer='normal'))
# Añadimos la capa de salida con 1
modelo.add( Dense(1, activation='relu'))

modelo.summary()

In [ ]:
# compilamos nuestro modelo
# Para compilar seleccione el optimizer adam dado que como nos comento es el que para estos casos el que más se recomienda, dado que es un optimizador estocastico y 
# que es un metodo con estimaciones adaptativas
modelo.compile(loss='mse',optimizer='adam', metrics=['mse','mae'])

In [ ]:
# Entrenamos nuestro modelo que para este caso he decidido hacerlo con 300 epochs y 50 de batch size
history=modelo.fit(X_train, d_train, validation_data=(X_test, d_test),batch_size=50, epochs=300)

In [ ]:
epochs_plot = range(1, len(history.history['mse'])+1, 1)

plt.plot(epochs_plot, history.history['mse'], 'r--',label='Evolucion del Mean square Error')
plt.plot(epochs_plot, history.history['mae'], 'r-',label='Precision')
plt.plot(epochs_plot, history.history['val_mse'], 'b--',label='Validacion')
plt.plot(epochs_plot, history.history['val_mae'], 'b-',label='Validacion')

plt.title('Performance de mi red neuronal')
plt.ylabel('Error')
plt.xlabel('Época')
plt.legend()
plt.figure()

In [ ]:
# vamos a hacer un prueba de nuestro modelo
# nos invetamos el caso de que tenemos un vino con estas características
X_new = np.array([[8.0,0.46,0.50,2.8,0.09300000000000001,19.0,95.0,0.99815,3.32,0.6,9.5]])
X_new = scaler_X.transform(X_new)
print(X_new)

In [ ]:
# Imprimimos el resultado que se obtiene al pasarle este caso, y nos devuelve el resultado de la calidad que obtendría dicho vino
d_new = modelo.predict(X_new)
# Tenbemos que tener un factor importante en este caso, y es que el resultado que nos da, no es resultado real, porque para ver el resultado real le tremos que desescalar
# de cara a ver la calidad autentica que obtendría
print(d_new)

In [ ]:
# Le aplicamos la transformación inversa para poder ver la calidad de este vino que le añadimos


d_new = scaler_d.inverse_transform(d_new)
# Tras aplicarle este inverse_transform ya estaría en la escala real.
print('Resultado redondeado = ',d_new.round(decimals=0, out=None))
print('Resultado con decimales = ',d_new)